# COMS W4705 Spring 24
## Homework 4 - Semantic Role Labelling with BERT

The goal of this assignment is to train and evaluate a PropBank-style semantic role labeling (SRL) system. Following (Collobert et al. 2011) and others, we will treat this problem as a sequence-labeling task. For each input token, the system will predict a B-I-O tag, as illustrated in the following example: 

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
|B-ARG1|I-ARG1|B-V|B-ARG2|I-ARG2|I-ARG2|I-ARG2|I-ARG2|O|O|O|O|O|O|O|
|||schedule.01|||||||||||||

Note that the same sentence may have multiple annotations for different predicates

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
|B-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|O|B-V|B-ARG2|I-ARG2|I-ARG2|B-ARGM-TMP|O|
||||||||||remove.01||||||

and not all predicates need to be verbs

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|    
|O|O|O|O|O|O|B-ARG1|B-V|O|O|O|O|O|O|O|
||||||||try.02||||||||

The SRL system will be implemented in [PyTorch](https://pytorch.org/). We will use BERT (in the implementation provided by the [Huggingface transformers](https://huggingface.co/docs/transformers/index) library) to compute contextualized token representations and a custom classification head to predict semantic roles. We will fine-tune the pretrained BERT model on the SRL task. 


### Overview of the Approach

The model we will train is pretty straightforward. Essentially, we will just encode the sentence with BERT, then take the contextualized embedding for each token and feed it into a classifier to predict the corresponding tag. 

Because we are only working on argument identification and labeling (not predicate identification), it is essentially that we tell the model where the predicate is. This can be accomplished in various ways. The approach we will choose here repurposes Bert's *segment embeddings*. 

Recall that BERT is trained on two input sentences, seperated by [SEP], and on a next-sentence-prediction objective (in addition to the masked LM objective). To help BERT comprehend which sentence a given token belongs to, the original BERT uses a segment embedding, using A for the first sentene, and B for the second sentence 2. 
Because we are labeling only a single sentence at a time, we can use the segment embeddings to indicate the predicate position instead: The predicate is labeled as segment B (1) and all other tokens will be labeled as segment A (0). 

<img src="https://github.com/daniel-bauer/4705-f23-hw5/blob/main/bert_srl_model.png?raw=true" width=400px>

## Setup: GCP, Jupyter, PyTorch, GPU

To make sure that PyTorch is available and can use the GPU,run the following cell which should return True. If it doesn't, make sure the GPU drivers and CUDA are installed correctly. 

GPU support is required for this assignment -- you will not be able to fine-tune BERT on a CPU. 

In [1]:
import torch
torch.cuda.is_available() 

True

## Dataset: Ontonotes 5.0 English SRL annotations

We will work with the English part of the [Ontonotes 5.0](https://catalog.ldc.upenn.edu/LDC2013T19) data. This is an extension of PropBank, using the same type of annotation. Ontonotes contains annotations other than predicate/argument structures, but we will use the PropBank style SRL annotations only. *Important*: This data set is provided to you for use in COMS 4705 only! Columbia is a subscriber to LDC and is allowed to use the data for educational purposes. However, you may not use the dataset in projects unrelated to Columbia teaching or research.

If you haven't done so already, you can download the data here: 


In [ ]:
! wget https://storage.googleapis.com/4705-bert-srl-data/ontonotes_srl.zip    

In [ ]:
! unzip ontonotes_srl.zip    

The data has been pre-processed in the following format. There are three files: 

`propbank_dev.tsv`	`propbank_test.tsv`	`propbank_train.tsv`

Each of these files is in a tab-separated value format. A single predicate/argument structure annotation consists of four rows. For example 

```
ontonotes/bc/cnn/00/cnn_0000.152.1
The     judge   scheduled       to      preside over    his     trial   was     removed from    the     case    today   /.
                schedule.01
B-ARG1  I-ARG1  B-V     B-ARG2  I-ARG2  I-ARG2  I-ARG2  I-ARG2  O       O       O       O       O       O       O
```

* The first row is a unique identifier (1st annotation of the 152nd sentence in the file ontonotes/bc/cnn/00/cnn_0000).
* The second row contains the tokens of the sentence (tab-separated). 
* The third row contains the probank frame name for the predicate (empty field for all other tokens). 
* The fourth row contains the B-I-O tag for each token. 

The file `rolelist.txt` contains a list of propbank BIO labels in the dataset (i.e. possible output tokens). This list has been filtered to contain only roles that appeared more than 1000 times in the training data. 
We will load this list and create mappings from numeric ids to BIO tags and back.

In [2]:
role_to_id = {}
with open("role_list.txt",'r') as f:
    role_list = [x.strip() for x in f.readlines()]
    role_to_id = dict((role, index) for (index, role) in enumerate(role_list))
    role_to_id['[PAD]'] = -100
    
    id_to_role = dict((index, role) for (role, index) in role_to_id.items())
    

Note that we are also mapping the '[PAD]' token to the value -100. This allows the loss function to ignore these tokens during training.

## Part 1 - Data Preparation

Before you can build the SRL model, you first need to preprocess the data. 


### 1.1 - Tokenization 

One challenge is that the pre-trained BERT model uses subword ("WordPiece") tokenization, but the Ontonotes data does not. Fortunately Huggingface transformers provides a tokenizer. 

In [3]:
from transformers import BertTokenizerFast 
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.tokenize("This is an unbelievably boring test sentence.")

['this',
 'is',
 'an',
 'un',
 '##bel',
 '##ie',
 '##va',
 '##bly',
 'boring',
 'test',
 'sentence',
 '.']

**TODO**: 
We need to be able to maintain the correct labels (B-I-O tags) for each of the subwords. 
Complete the following function that takes a list of tokens and a list of B-I-O labels of the same length as parameters, and returns a new token / label pair, as illustrated in the following example. 


```
>>> tokenize_with_labels("the fancyful penguin devoured yummy fish .".split(), "B-ARG0 I-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 O".split(), tokenizer)
(['the',
  'fancy',
  '##ful',
  'penguin',
  'dev',
  '##oured',
  'yu',
  '##mmy',
  'fish',
  '.'],
 ['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-V',
  'I-V',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'])

```

To approach this problem, iterate through each word/label pair in the sentence. Call the tokenizer on the word. This may result in one or more tokens. Create the correct number of labels to match the number of tokens. Take care to not generate multiple B- tokens. 


This approach is a bit slower than tokenizing the entire sentence, but is necessary to produce proper input tokenization for the pre-trained BERT model, and the matching target labels. 

In [4]:
def tokenize_with_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. 
    """

    tokenized_sentence = []
    labels = []
    
    for word, label in zip(sentence, text_labels):
        subwords = tokenizer.tokenize(word)  # Tokenizing the word into subwords
        
        # Preparing new labels for subwords
        if len(subwords) > 1:
            # Ensuring only 'B-' labels are converted to 'I-' for subsequent subwords
            new_labels = [label if i == 0 else (label.replace('B-', 'I-') if 'B-' in label else label) for i in range(len(subwords))]
        else:
            new_labels = [label]
        
        tokenized_sentence.extend(subwords)
        labels.extend(new_labels)  # Extending labels to match the new subwords

    return tokenized_sentence, labels


In [5]:
tokenize_with_labels("the fancyful penguin devoured yummy fish .".split(), "B-ARG0 I-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 O".split(), tokenizer)

(['the',
  'fancy',
  '##ful',
  'penguin',
  'dev',
  '##oured',
  'yu',
  '##mmy',
  'fish',
  '.'],
 ['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-V',
  'I-V',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'])

In [6]:
# Test Case
sentence = "The manager approved the proposal quickly but cautiously declined the invitation .".split()


labels = "B-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 B-ARGM-ADV O B-ARGM-ADV B-V B-ARG1 I-ARG1 O".split()



tokenized_sentence, tokenized_labels = tokenize_with_labels(sentence, labels, tokenizer)
print("Tokenized Sentence:", tokenized_sentence)
print("Tokenized Labels:  ", tokenized_labels)

Tokenized Sentence: ['the', 'manager', 'approved', 'the', 'proposal', 'quickly', 'but', 'cautiously', 'declined', 'the', 'invitation', '.']
Tokenized Labels:   ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-ADV', 'O', 'B-ARGM-ADV', 'B-V', 'B-ARG1', 'I-ARG1', 'O']


In [7]:
# Test Case

def test_tokenize_and_label_alignment():
    sample_sentence = ["The", "judge", "scheduled", "the", "trial"]
    sample_labels = ["O", "B-ARG1", "B-V", "O", "O"]
    tokenized_sentence, labels = tokenize_with_labels(sample_sentence, sample_labels, tokenizer)
    

    assert len(tokenized_sentence) == len(labels), "Length of tokenized sentence should match labels"
    

    assert 'B-V' in labels, "Predicate label B-V should exist in labels"
    

    assert labels[sample_sentence.index("scheduled")] == 'B-V', "Label 'B-V' should be assigned to 'scheduled'"
    
    print("Test passed for tokenization and label alignment.")

test_tokenize_and_label_alignment()

Test passed for tokenization and label alignment.


In [8]:
# Test Case

sample_sentence = ["Your", "example", "sentence"]
sample_labels = ["O", "B-V", "O"]
tokens, labels = tokenize_with_labels(sample_sentence, sample_labels, tokenizer)
print("Debugging Input Processing:")
print("Tokens:", tokens)
print("Labels:", labels)


Debugging Input Processing:
Tokens: ['your', 'example', 'sentence']
Labels: ['O', 'B-V', 'O']


In [9]:
# Test Case Complex

def test_complex_cases(tokenizer):
    tests = [
        ("He believes the quick resolution of the conflict won't happen soon.", 
         "B-ARG0 O B-ARG1 I-ARG1 I-ARG1 B-ARG1 I-ARG1 I-ARG1 O B-ARGM-TMP O".split()),
        ("They will renegotiate the contract effectively and swiftly by tomorrow.", 
         "B-ARG0 O B-V B-ARG1 I-ARG1 B-ARGM-MNR B-ARGM-MNR O B-ARGM-TMP O".split())
    ]
    
    for sentence, labels in tests:
        tokenized_sentence, tokenized_labels = tokenize_with_labels(sentence.split(), labels, tokenizer)
        print("Sentence:", sentence)
        print("Tokenized Sentence:", tokenized_sentence)
        print("Expected Labels: ", labels)
        print("Tokenized Labels:  ", tokenized_labels)
        print("---")

test_complex_cases(tokenizer)

Sentence: He believes the quick resolution of the conflict won't happen soon.
Tokenized Sentence: ['he', 'believes', 'the', 'quick', 'resolution', 'of', 'the', 'conflict', 'won', "'", 't', 'happen', 'soon', '.']
Expected Labels:  ['B-ARG0', 'O', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'B-ARGM-TMP', 'O']
Tokenized Labels:   ['B-ARG0', 'O', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'O', 'O', 'B-ARGM-TMP', 'O', 'O']
---
Sentence: They will renegotiate the contract effectively and swiftly by tomorrow.
Tokenized Sentence: ['they', 'will', 'rene', '##go', '##tia', '##te', 'the', 'contract', 'effectively', 'and', 'swiftly', 'by', 'tomorrow', '.']
Expected Labels:  ['B-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'B-ARGM-MNR', 'O', 'B-ARGM-TMP', 'O']
Tokenized Labels:   ['B-ARG0', 'O', 'B-V', 'I-V', 'I-V', 'I-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'B-ARGM-MNR', 'O', 'B-ARGM-TMP', 'O', 'O']
---


### 1.2 Loading the Dataset

Next, we are creating a PyTorch [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class. This class acts as a contained for the training, development, and testing data in memory. You should already be familiar with Datasets and Dataloaders from homework 3. 

1.2.1 **TODO**: Write the \_\_init\_\_(self, filename) method that reads in the data from a data file (specified by the filename).

For each annotation you start with  the tokens in the sentence, and the BIO tags. Then you need to create the following 

1. call the `tokenize_with_labels` function to tokenize the sentence.
2. Add the (token, label) pair to the self.items list. 

1.2.2 **TODO**: Write the \_\_len\_\_(self) method that returns the total number of items. 

1.2.3 **TODO**: Write the \_\_getitem\_\_(self, k) method that returns a single item in a format BERT will understand. 
* We need to process the sentence by adding "\[CLS\]" as the first token and "\[SEP\]" as the last token. The need to pad the token sequence to 128 tokens using the "\[PAD\]" symbol. This needs to happen both for the inputs (sentence token sequence) and outputs (BIO tag sequence).
* We need to create an *attention mask*, which is a sequence of 128 tokens indicating the actual input symbols (as a 1) and \[PAD\] symbols (as a 0).
* We need to create a *predicate indicator* mask, which is a sequence of 128 tokens with at most one 1, in the position of the "B-V" tag. All other entries should be 0. The model will use this information to understand where the predicate is located. 

* Finally, we need to convert the token and tag sequence into numeric indices. For the tokens, this can be done using the `tokenizer.convert_tokens_to_ids` method. For the tags, use the `role_to_id` dictionary. 
Each sequence must be a pytorch tensor of shape (1,128). You can convert a list of integer values like this `torch.tensor(token_ids, dtype=torch.long)`.

To keep everything organized, we will return a dictionary in the following format 

```
{'ids': token_tensor,
 'targets': tag_tensor,
 'mask': attention_mask_tensor,
 'pred': predicate_indicator_tensor}
```


(Hint: To debug these, read in the first annotation only / the first few annotations) 


In [10]:
from torch.utils.data import Dataset, DataLoader


class SrlData(Dataset):
    def __init__(self, filename):
        super(SrlData, self).__init__()
        
        self.max_len = 128 # the max number of tokens inputted to the transformer. 
        
        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)
        
        self.items = []

        with open(filename, 'r') as file:
            while True:
                header = file.readline().strip()
                if not header:
                    break  # End of file
                tokens = file.readline().strip().split()
                predicate = file.readline().strip()  
                labels = file.readline().strip().split()
                
                # Tokenizing and aligning labels with tokens
                tokenized_text, labels = tokenize_with_labels(tokens, labels, self.tokenizer)
                if len(tokenized_text) > self.max_len - 2:  # Account for [CLS] and [SEP]
                    tokenized_text = tokenized_text[:self.max_len-2]
                    labels = labels[:self.max_len-2]

                tokenized_text = ['[CLS]'] + tokenized_text + ['[SEP]']
                labels = ['O'] + labels + ['O']  # Padding labels correspondingly

                self.items.append((tokenized_text, labels))
        
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, k):
        tokens, labels = self.items[k]
        
        # Padding tokens and labels to max_len
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        token_ids += [self.tokenizer.pad_token_id] * (self.max_len - len(token_ids))
        attention_mask = [1] * len(tokens) + [0] * (self.max_len - len(tokens))
        
        label_ids = [role_to_id[label] if label in role_to_id else role_to_id['O'] for label in labels]
        label_ids += [role_to_id['O']] * (self.max_len - len(label_ids))
        
        predicate_indicator = [1 if label == 'B-V' else 0 for label in labels] + [0] * (self.max_len - len(labels))
        
        return {
            'ids': torch.tensor(token_ids, dtype=torch.long),
            'mask': torch.tensor(attention_mask, dtype=torch.long),
            'targets': torch.tensor(label_ids, dtype=torch.long),
            'pred': torch.tensor(predicate_indicator, dtype=torch.long)
        }

In [46]:
# Reading the training data takes a while for the entire data because we preprocess all data offline
data = SrlData("propbank_train.tsv")

In [16]:
# Checking if the data are being properly read

# Printing 5 samples from the dataset
for i in range(5):  
    sample = data[i]
    print(f"Sample {i}:")
    print("Tokens:", data.tokenizer.convert_ids_to_tokens(sample['ids']))
    print("Token IDs:", sample['ids'])
    print("Labels:", sample['targets'])
    print("Attention Mask:", sample['mask'])
    print("Predicate Indicator:", sample['pred'])
    print("\n")


Sample 0:
Tokens: ['[CLS]', 'we', 'respectful', '##ly', 'invite', 'you', 'to', 'watch', 'a', 'special', 'edition', 'of', 'across', 'china', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [9]:
# Checking if all items have the correct length

for i, item in enumerate(data):
    assert len(item['ids']) == data.max_len, f"Error in token length at index {i}"
    assert len(item['targets']) == data.max_len, f"Error in label length at index {i}"
    assert len(item['mask']) == data.max_len, f"Error in mask length at index {i}"
    assert len(item['pred']) == data.max_len, f"Error in predicate indicator length at index {i}"

print("All samples are correctly formatted.")


All samples are correctly formatted.


## 2. Model Definition

In [11]:
from torch.nn import Module, Linear, CrossEntropyLoss
from transformers import BertModel

We will define the pyTorch model as a subclass of the [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) class. The code for the model is provided for you. It may help to take a look at the documentation to remind you of how Module works. Take a look at how the huggingface BERT model simply becomes another sub-module. 

In [13]:
class SrlModel(Module):
    
    def __init__(self):
        
        super(SrlModel, self).__init__()
        
        self.encoder = BertModel.from_pretrained("bert-base-uncased")
    
        # The following two lines would freeze the BERT parameters and allow us to train the classifier by itself.
        # We are fine-tuning the model, so you can leave this commented out!
        # for param in self.encoder.parameters():
        #    param.requires_grad = False
        
        # The linear classifier head, see model figure in the introduction. 
        self.classifier = Linear(768, len(role_to_id))
                
        
    def forward(self, input_ids, attn_mask, pred_indicator):
    
        # This defines the flow of data through the model 
    
        # Note the use of the "token type ids" which represents the segment encoding explained in the introduction. 
        # In our segment encoding, 1 indicates the predicate, and 0 indicates everything else. 
        bert_output =  self.encoder(input_ids=input_ids, attention_mask=attn_mask, token_type_ids=pred_indicator)

        enc_tokens = bert_output[0] # the result of encoding the input with BERT
        logits = self.classifier(enc_tokens) #feed into the classification layer to produce scores for each tag.
        
        # Note that we are only interested in the argmax for each token, so we do not have to normalize 
        # to a probability distribution using softmax. The CrossEntropyLoss loss function takes this into account.
        # It essentially computes the softmax first and then computes the negative log-likelihood for the target classes. 
        return logits         

In [14]:
model = SrlModel().to('cuda') # create new model and store weights in GPU memory

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Now we are ready to try running the model with just a single input example to check if it is working correctly. Clearly it has not been trained, so the output is not what we expect. But we can see what the loss looks like for an initial sanity check. 

**TODO**: 
* Take a single data item from the dev set, as provided by your Dataset class defined above. Obtain the input token ids, attention mask, predicate indicator mask, and target labels. 
* Run the model on the ids, attention mask, and predicate mask like this: 

In [50]:
 
single_data_item = data[0] 

# Inputs for the model
ids = single_data_item['ids'].unsqueeze(0).to('cuda')  
mask = single_data_item['mask'].unsqueeze(0).to('cuda')
pred = single_data_item['pred'].unsqueeze(0).to('cuda')
  

# Initializing the model
outputs = model(ids, mask, pred)

In [51]:
print("Model outputs (logits):", outputs)

Model outputs (logits): tensor([[[-0.3564,  0.0865, -0.3082,  ..., -0.4487,  0.1145,  0.1511],
         [ 0.1865, -0.1831,  0.2159,  ..., -0.3317, -0.3368, -0.1022],
         [-0.1480, -0.0322,  0.1638,  ..., -0.0050, -0.1539,  0.2404],
         ...,
         [-0.0883,  0.0278, -0.2977,  ...,  0.1163, -0.0961, -0.3726],
         [-0.1381,  0.0739, -0.2802,  ...,  0.1242, -0.1297, -0.3620],
         [-0.0196,  0.0319, -0.2335,  ...,  0.1568, -0.1574, -0.3519]]],
       device='cuda:0', grad_fn=<AddBackward0>)


**TODO**: 
Compute the loss on this one item only.
The initial loss should be close to -ln(1/num_labels)

Without training we would assume that all labels for each token (including the target label) are equally likely, so the negative log probability for the targets should be approximately $$-\ln(\frac{1}{\text{num_labels}}).$$ This is what the loss function should return on a single example. This is a good sanity check to run for any multi-class prediction problem. 

In [52]:
import math

In [53]:

num_labels = len(role_to_id)
expected_initial_loss = -math.log(1 / num_labels, math.e)


targets = single_data_item['targets'].unsqueeze(0).to('cuda')
loss_function = CrossEntropyLoss(ignore_index=role_to_id['[PAD]'], reduction='mean')
loss = loss_function(outputs.view(-1, num_labels), targets.view(-1))  


print("Initial loss:", loss.item())
print("Expected initial loss:", expected_initial_loss)



Initial loss: 3.9228010177612305
Expected initial loss: 3.970291913552122


In [54]:
print("Raw logits for the first token:", outputs[:, 0, :])

Raw logits for the first token: tensor([[-0.3564,  0.0865, -0.3082, -0.0727, -0.1110,  0.0343,  0.3094, -0.1314,
         -0.2484,  0.3390,  0.1887,  0.2909,  0.5158,  0.2699, -0.3522, -0.5078,
         -0.2392, -0.4452, -0.3743, -0.4032,  0.0773,  0.7311, -0.0976, -0.3880,
          0.4402, -0.0724,  0.0883,  0.0406,  0.0079,  0.5992,  0.1019, -0.0920,
          0.5086, -0.2860, -0.1027, -0.1548, -0.4966, -0.3372, -0.4492, -0.6973,
          0.5503, -0.3454,  0.1766, -0.0514,  0.0873, -0.1378,  0.3647,  0.1337,
         -0.4053,  0.1446, -0.4487,  0.1145,  0.1511]], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [55]:
cls_label_idx = torch.argmax(outputs[:, 0, :], dim=-1)
cls_label = id_to_role[cls_label_idx.item()]
print("Most likely label for [CLS]:", cls_label)

Most likely label for [CLS]: B-ARGM-NEG


**TODO**: At this point you should also obtain the actual predictions by taking the argmax over each position.
The result should look something like this (values will differ).

```
tensor([[ 1,  4,  4,  4,  4,  4,  5, 29, 29, 29,  4, 28,  6, 32, 32, 32, 32, 32,
         32, 32, 30, 30, 32, 30, 32,  4, 32, 32, 30,  4, 49,  4, 49, 32, 30,  4,
         32,  4, 32, 32,  4,  2,  4,  4, 32,  4, 32, 32, 32, 32, 30, 32, 32, 30,
         32,  4,  4, 49,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  6,  6, 32, 32,
         30, 32, 32, 32, 32, 32, 30, 30, 30, 32, 30, 49, 49, 32, 32, 30,  4,  4,
          4,  4, 29,  4,  4,  4,  4,  4,  4, 32,  4,  4,  4, 32,  4, 30,  4, 32,
         30,  4, 32,  4,  4,  4,  4,  4, 32,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4]], device='cuda:0')
```

Then use the id_to_role dictionary to decode to actual tokens. 

```
['[CLS]', 'O', 'O', 'O', 'O', 'O', 'B-ARG0', 'I-ARG0', 'I-ARG0', 'I-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG1', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'O', 'I-ARGM-TMP', 'O', 'I-ARGM-TMP', 'I-ARG2', 'I-ARG1', 'O', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'O', '[SEP]', 'O', 'O', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'O', 'O', 'I-ARGM-TMP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG1', 'B-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG2', 'I-ARG1', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'O', 'O', 'O', 'O', 'I-ARG0', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ARG2', 'O', 'O', 'O', 'I-ARG2', 'O', 'I-ARG1', 'O', 'I-ARG2', 'I-ARG1', 'O', 'I-ARG2', 'O', 'O', 'O', 'O', 'O', 'I-ARG2', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
```

For now, just make sure you understand how to do this for a single example. Later, you will write a more formal function to do this once we have trained the model. 

In [56]:
predictions = torch.argmax(outputs, dim=-1)  # Applying argmax to get the most likely label index for each position

# Decoding these indices to their corresponding tags using 'id_to_role'
decoded_labels = [id_to_role[idx.item()] for idx in predictions[0]]  

print("Predictions indices tensor:", predictions)
print("Decoded labels:", decoded_labels)


Predictions indices tensor: tensor([[21, 12, 12, 24, 21, 40, 21,  4, 24, 32, 50, 43, 24, 24, 52, 50, 32, 32,
         32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 21, 11, 21, 32,  9,
         21, 41,  9, 24, 24, 24,  7, 24, 32, 32, 32, 32, 32, 21, 32, 32, 32, 32,
         32, 32, 32, 32, 41, 11, 11, 21, 32,  9,  9, 50, 50, 24, 24, 32, 17, 32,
         32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 21, 21, 11,
         21, 32,  9,  9, 50, 50, 24, 24, 24, 32, 32, 32, 32, 32, 32, 32, 32, 32,
         32, 32, 32, 32, 32, 32, 32, 32, 21, 21,  7, 32, 50,  9, 26, 17,  3, 32,
         32, 32]], device='cuda:0')
Decoded labels: ['B-ARGM-NEG', 'B-ARGM-ADV', 'B-ARGM-ADV', 'B-ARGM-PRR', 'B-ARGM-NEG', 'I-ARGM-EXT', 'B-ARGM-NEG', 'O', 'B-ARGM-PRR', 'I-ARG2', 'I-ARGM-CXN', 'I-ARGM-MNR', 'B-ARGM-PRR', 'B-ARGM-PRR', 'I-V', 'I-ARGM-CXN', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2'

## 3. Training loop

pytorch provides a DataLoader class that can be wrapped around a Dataset to easily use the dataset for training. The DataLoader allows us to easily adjust the batch size and shuffle the data. 

In [18]:
from torch.utils.data import DataLoader
loader = DataLoader(data, batch_size = 32, shuffle = True)

The following cell contains the main training loop. The code should work as written and report the loss after each batch, cumulative average loss after each 100 batches, and print out the final average loss after the epoch. 

**TODO**: Modify the training loop below so that it also computes the accuracy for each batch and reports the 
average accuracy after the epoch. 
The accuracy is the number of correctly predicted token labels out of the number of total predictions. 
Make sure you exclude [PAD] tokens, i.e. tokens for which the target label is -100. It's okay to include [CLS] and [SEP] in the accuracy calculation. 

In [22]:
from torch.nn import CrossEntropyLoss
import torch.optim as optim
from tqdm import tqdm


loss_function = CrossEntropyLoss(ignore_index=-100, reduction='mean')
LEARNING_RATE = 1e-05
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
device = 'cuda'


def train(loader, model):
    """
    Train the model for one epoch, track and report batch loss, cumulative average loss,
    and accuracy while excluding padding tokens from accuracy calculations.
    """
    model.train()  # Set the model to training mode
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    total_correct = 0
    total_tokens = 0  # Total non-PAD tokens processed for accuracy calculation

    # Using tqdm for a visual progress bar
    for idx, batch in enumerate(tqdm(loader, desc="Training Progress")):
        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)
        targets = batch['targets'].to(device, dtype=torch.long)
        pred_mask = batch['pred'].to(device, dtype=torch.long)

        # Forward pass
        logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
        loss = loss_function(logits.transpose(2, 1), targets)
        tr_loss += loss.item()
        
        
        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        # Accuracy
        predictions = torch.argmax(logits, dim=2)
        correct = (predictions == targets) & (targets != -100)
        total_correct += correct.sum().item()
        total_tokens += (targets != -100).sum().item()

        # Log loss every 100 batches
        if (idx + 1) % 100 == 0:
            curr_avg_loss = tr_loss / nb_tr_steps
            curr_avg_acc = total_correct / total_tokens * 100
            print(f"Batch {idx + 1}, Current average loss: {curr_avg_loss}, Current average accuracy: {curr_avg_acc:.2f}%")
        
        # Backward and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Final metrics for the epoch
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accuracy = total_correct / total_tokens * 100 if total_tokens > 0 else 0
    print(f"Final Epoch Loss: {epoch_loss:.4f}, Final Epoch Accuracy: {epoch_accuracy:.2f}%")




Now let's train the model for one epoch. This will take a while (up to a few hours). 

In [23]:
train(loader, model)

Training Progress:   7%|▋         | 99/1365 [01:16<15:58,  1.32it/s]

Batch 100, Current average loss: 1.0329221260547639, Current average accuracy: 84.91%


Training Progress:  15%|█▍        | 199/1365 [02:33<14:47,  1.31it/s]

Batch 200, Current average loss: 0.7127039429545402, Current average accuracy: 88.16%


Training Progress:  22%|██▏       | 299/1365 [03:50<13:40,  1.30it/s]

...

Batch 1200, Current average loss: 0.33521903516103824, Current average accuracy: 92.56%


Training Progress:  95%|█████████▌| 1299/1365 [16:36<00:50,  1.31it/s]

Batch 1300, Current average loss: 0.3227103732698239, Current average accuracy: 92.78%


Training Progress: 100%|██████████| 1365/1365 [17:27<00:00,  1.30it/s]

Final Epoch Loss: 0.3155, Final Epoch Accuracy: 92.90%


In my experiments, I found that two epochs are needed for good performance. 

In [24]:
train(loader, model) 

Training Progress:   7%|▋         | 99/1365 [01:16<15:40,  1.35it/s]

Batch 100, Current average loss: 0.15658706426620483, Current average accuracy: 95.85%


Training Progress:  15%|█▍        | 199/1365 [02:33<14:40,  1.32it/s]

Batch 200, Current average loss: 0.1518608769029379, Current average accuracy: 95.93%


Training Progress:  22%|██▏       | 299/1365 [03:49<13:35,  1.31it/s]

...

Batch 1200, Current average loss: 0.12651143576639393, Current average accuracy: 96.59%


Training Progress:  95%|█████████▌| 1299/1365 [16:34<00:50,  1.31it/s]

Batch 1300, Current average loss: 0.12440695476073485, Current average accuracy: 96.64%


Training Progress: 100%|██████████| 1365/1365 [17:24<00:00,  1.31it/s]

Final Epoch Loss: 0.1231, Final Epoch Accuracy: 96.67%


I ended up with a training loss of about 0.19 and a training accuracy of 0.94. Specific values may differ. 

At this point, it's a good idea to save the model (or rather the parameter dictionary) so you can continue evaluating the model without having to retrain. 

In [15]:
torch.save(model.state_dict(), "srl_model_fulltrain_2epoch_finetune_1e-05.pt")

## 4. Decoding

In [16]:
# Optional step: If you stopped working after part 3, first load the trained model 

model = SrlModel().to('cuda') 
model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))
model = model.to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**TODO (this is the fun part)**: Now that we have a trained model, let's try labeling an unseen example sentence. Complete the functions decode_output and label_sentence below. decode_output takes the logits returned by the model, extracts the argmax to obtain the label predictions for each token, and then translate the result into a list of string labels. 

label_sentence takes a list of input tokens and a predicate index, prepares the model input, call the model and then call decode_output to produce a final result. 

Note that you have already implemented all components necessary (preparing the input data from the token list and predicate index, decoding the model output). But now you are putting it together in one convenient function. 

In [17]:
tokens = "A U. N. team spent an hour inside the hospital , where it found evident signs of shelling and gunfire .".split()

In [18]:
def decode_output(logits): # it will be useful to have this in a separate function later on
    """
    Given the model output logits, convert logits to label indices and translate to string labels.
    """
    # Converting logits to predicted label indices
    predictions = torch.argmax(logits, dim=-1).squeeze(0)  # Remove batch dimension assuming batch size of 1
    # Translating indices to labels using global id_to_role dictionary
    labels = [id_to_role[idx] for idx in predictions.cpu().numpy()]
    return labels

    

In [19]:
def label_sentence(tokens, pred_idx):
    
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True, max_length=128)
    input_ids = inputs['input_ids'].to('cuda')
    attention_mask = inputs['attention_mask'].to('cuda')

    pred_mask = torch.zeros_like(input_ids)
    pred_mask[0, pred_idx + 1] = 1  # Adjusting for [CLS]
    pred_mask = pred_mask.to('cuda')

    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask, pred_mask)
        
    labels = decode_output(logits)
    

    return labels

        

In [20]:
# Now you should be able to run

pred_idx = 13  # Index for 'found'
label_test = label_sentence(tokens, pred_idx)
output = list(zip(tokens, label_test))
for token_label in output:
    print(token_label)


('A', 'B-ARGM-LVB')
('U.', 'I-ARG1')
('N.', 'I-ARGM-GOL')
('team', 'I-ARGM-GOL')
('spent', 'I-ARGM-EXT')
('an', 'B-ARGM-TMP')
('hour', 'I-ARGM-PRP')
('inside', 'I-ARGM-GOL')
('the', 'I-ARGM-GOL')
('hospital', 'I-ARGM-MNR')
(',', 'I-ARGM-GOL')
('where', 'I-ARG0')
('it', 'B-ARGM-CAU')
('found', 'I-ARGM-EXT')
('evident', 'I-ARGM-PRP')
('signs', 'I-ARGM-GOL')
('of', 'I-ARGM-PRP')
('shelling', 'I-ARGM-PRP')
('and', 'I-ARGM-MNR')
('gunfire', 'I-ARGM-MNR')
('.', 'I-ARGM-GOL')


The expected output is somethign like this: 
```   
 ('A', 'O'),
 ('U.', 'O'),
 ('N.', 'O'),
 ('team', 'O'),
 ('spent', 'O'),
 ('an', 'O'),
 ('hour', 'O'),
 ('inside', 'O'),
 ('the', 'B-ARGM-LOC'),
 ('hospital', 'I-ARGM-LOC'),
 (',', 'O'),
 ('where', 'B-ARGM-LOC'),
 ('it', 'B-ARG0'),
 ('found', 'B-V'),
 ('evident', 'B-ARG1'),
 ('signs', 'I-ARG1'),
 ('of', 'I-ARG1'),
 ('shelling', 'I-ARG1'),
 ('and', 'I-ARG1'),
 ('gunfire', 'I-ARG1'),
 ('.', 'O'),
```


### 5. Evaluation 1: Token-Based Accuracy
We want to evaluate the model on the dev or test set. 

In [21]:
dev_data = SrlData("propbank_dev.tsv") # Takes a while because we preprocess all data offline

In [22]:
from torch.utils.data import DataLoader
loader = DataLoader(dev_data, batch_size = 1, shuffle = False)

In [24]:
# Optional: Load the model again if you stopped working prior to this step. 
model = SrlModel()
model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))
model = model.to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
print("Testing basic execution")


Testing basic execution


**TODO**: Complete the evaluate_token_accuracy function below. The function should iterate through the items in the data loader (see training loop in part 3). Run the model on each sentence/predicate pair and extract the predictions.

For each sentence, count the correct predictions and the total predictions. Finally, compute the accuracy as #correct_predictions / #total_predictions

Careful: You need to filter out the padded positions ([PAD] target tokens), as well as [CLS] and [SEP]. It's okay to include [B-V] in the count though. 

In [ ]:
def evaluate_token_accuracy(model, loader, device='cuda'):
    print("Function start")
    model.eval()  

    total_correct = 0
    total_predictions = 0

    for idx, batch in enumerate(loader):
        print(f"Processing batch {idx+1}")
        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)
        targets = batch['targets'].to(device, dtype=torch.long)
        pred_mask = batch['pred'].to(device, dtype=torch.long)

        # Prediction using the model
        with torch.no_grad():
            logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
        
        print("Model has made predictions.")
        
        # Getting the predicted labels by taking the argmax
        predictions = torch.argmax(logits, dim=2)

        # Calculating the mask to exclude [PAD], [CLS], and [SEP] tokens
        valid_mask = (targets != role_to_id['[PAD]']) & (targets != role_to_id['[CLS]']) & (targets != role_to_id['[SEP]'])

        correct = torch.sum((predictions == targets) & valid_mask)
        total_correct += correct.item()

        total_predictions += valid_mask.sum().item()

    # Computing accuracy
    accuracy = total_correct / total_predictions if total_predictions > 0 else 0
    print(f"Accuracy: {accuracy:.4f}")


evaluate_token_accuracy(model, loader)




Function start
Processing batch 1
Model has made predictions.
Processing batch 2
Model has made predictions.
Processing batch 3
Model has made predictions.
Processing batch 4
Model has made predictions.
Processing batch 5
Model has made predictions.
Processing batch 6
Model has made predictions.
Processing batch 7
Model has made predictions.
Processing batch 8
Model has made predictions.
Processing batch 9
Model has made predictions.
Processing batch 10
Model has made predictions.
Processing batch 11
Model has made predictions.
Processing batch 12
Model has made predictions.
Processing batch 13
Model has made predictions.
Processing batch 14
Model has made predictions.
Processing batch 15
Model has made predictions.
Processing batch 16
Model has made predictions.
Processing batch 17
Model has made predictions.
Processing batch 18
Model has made predictions.
Processing batch 19
Model has made predictions.
Processing batch 20
Model has made predictions.
Processing batch 21
Model has made

In [72]:
print(f"Number of batches: {len(loader)}")


Number of batches: 43665


In [73]:
for idx, batch in enumerate(loader):
    if idx == 0:  
        print(batch)


{'ids': tensor([[ 101, 1999, 1996, 2621, 1997, 2384, 1010, 1037, 3861, 2008, 2111, 2031,
         2146, 2042, 2559, 2830, 2000, 2318, 8361, 2007, 6075, 1999, 2536, 2350,
         4291, 4290, 2865, 1012,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 

### 6. Span-Based evaluation 

While the accuracy score in part 5 is encouraging, an accuracy-based evaluation is problematic for two reasons. First, most of the target labels are actually O. Second, it only tells us that per-token prediction works, but does not directly evaluate the SRL performance. 

Instead, SRL systems are typically evaluated on micro-averaged precision, recall, and F1-score for predicting labeled spans. 

More specifically, for each sentence/predicate input, we run the model, decode the output, and extract a set of labeled spans (from the output and the target labels). These spans are (i,j,label) tuples.  

We then compute the true_positives, false_positives, and false_negatives based on these spans. 

In the end, we can compute 

* Precision:  true_positive / (true_positives + false_positives)  , that is the number of correct spans out of all predicted spans. 

* Recall: true_positives / (true_positives + false_negatives) , that is the number of correct spans out of all target spans. 

* F1-score:   (2 * precision * recall) / (precision + recall)


For example, consider 

| |[CLS]|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|--||---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
||0|1|2|3|4|5|6|7|8|9|1O|11|12|13|14|15|
|target|[CLS]|B-ARG1|I-ARG1|B-V|B-ARG2|I-ARG2|I-ARG2|I-ARG2|I-ARG2|O|O|O|O|O|O|O|
|prediction|[CLS]|B-ARG1|I-ARG1|B-V|I-ARG2|I-ARG2|O|O|O|O|O|O|O|O|B-ARGM-TMP|O|

The target spans are (1,2,"ARG1"), and (4,8,"ARG2").

The predicted spans would be (1,2,"ARG1"), (14,14,"ARGM-TMP"). Note that in the prediction, there is no proper ARG2 span because we are missing the B-ARG2 token, so this span should not be created. 

So for this sentence we woudl get: true_positives: 1 false_positives: 1 false_negatives: 1

*TODO*: Complete the function evaluate_spans that performs the span-based evaluation on the given model and data loader. You can use the provided extract_spans function, which returns the spans as a dictionary. For example
{(1,2): "ARG1", (4,8):"ARG2"} 

In [27]:
def extract_spans(labels):
    spans = {} # map (start,end) ids to label
    current_span_start = 0
    current_span_type = ""
    inside = False
    for i, label in enumerate(labels):
        if label.startswith("B"):            
            if inside: 
                if current_span_type != "V":
                    spans[(current_span_start,i)] = current_span_type            
            current_span_start = i
            current_span_type = label[2:]
            inside = True
        elif inside and label.startswith("O"):
            if current_span_type != "V":
                spans[(current_span_start,i)] = current_span_type
            inside = False
        elif inside and label.startswith("I") and label[2:] != current_span_type:            
            if current_span_type != "V":
                spans[(current_span_start,i)] = current_span_type
            inside = False
    return spans
                        

In [ ]:
def evaluate_spans(model, loader, device='cuda'):
    model.eval()
    total_tp = 0
    total_fp = 0
    total_fn = 0
​
    for idx, batch in enumerate(loader):
        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)
        targets = batch['targets'].to(device, dtype=torch.long)
        pred_mask = batch['pred'].to(device, dtype=torch.long)
​
        with torch.no_grad():
            logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
            predictions = torch.argmax(logits, dim=2).cpu().numpy()
            targets = targets.cpu().numpy()
​
        for i in range(predictions.shape[0]):  # processing each item in the batch
            pred_spans = extract_spans(predictions[i], id_to_role)
            target_spans = extract_spans(targets[i], id_to_role)
​
            for span, label in pred_spans.items():
                if span in target_spans and target_spans[span] == label:
                    total_tp += 1
                else:
                    total_fp += 1
​
            for span, label in target_spans.items():
                if span not in pred_spans or pred_spans.get(span) != label:
                    total_fn += 1
​
    total_p = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    total_r = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    total_f = (2 * total_p * total_r) / (total_p + total_r) if (total_p + total_r) > 0 else 0
​
    print(f"Overall P: {total_p:.3f}  Overall R: {total_r:.3f}  Overall F1: {total_f:.3f}")
​
​
evaluate_spans(model, loader)
  

In my evaluation, I got an F score of 0.82  (which slightly below the state-of-the art in 2018)

### OPTIONAL: 

Repeat the span-based evaluation, but print out precision/recall/f1-score for each role separately.